# Routing
llm will be choosing which path will it take.

In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

True

In [5]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
embedding_path = os.getenv("EMBEDDING_PATH")
embedding_path

'D:\\Generative AI\\Gen AI Language\\local LLM\\local embedding\\sentence-transformers-local\\all-MiniLM-L6-v2'

In [6]:
from sentence_transformers import SentenceTransformer
emmbedding_model = SentenceTransformer(embedding_path)
emmbedding_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [7]:
from langchain_groq import ChatGroq

llm = ChatGroq()
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002565766EFC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002565752ADE0>, model_kwargs={}, groq_api_key=SecretStr('**********'))

# Logical and Semantic Routing

In [10]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

class RouteQuery(BaseModel):
    '''Route a user query to the most relevant source.'''
    datasource: Literal["youtube", "linkedin", "twitter"] = Field(
        ..., description="Given a user question choose which social media platform to query."
    )

structured_llm = llm.with_structured_output(RouteQuery)

system = '''You are an expert at routing a user question to the appropriate social media platform.'''

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}")
    ]
)

router = prompt | structured_llm

In [11]:
router

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert at routing a user question to the appropriate social media platform.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002565766EFC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002565752ADE0>, model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'RouteQuery', 'description': 'Route a user query to the most relevant source.', 'parameters': {'type': 'object', 'properties': {'datasource': {'description': 'Given a user question 

In [12]:
question = '''
I am looking for a tutorial on how to code.
'''

result = router.invoke({"question": question})

result

RouteQuery(datasource='youtube')

In [13]:
result.datasource

'youtube'

In [14]:
def choose_route(result):
    if "youtube" in result.datasource:
        return "https://www.youtube.com"
    elif "linkedin" in result.datasource:
        return "https://www.linkedin.com"
    else:
        return "https://ww.x.com"

In [18]:
choose_route(result)

'https://www.youtube.com'

In [20]:
choose_route(router.invoke({"question": "I am looking for a job."}))

'https://www.linkedin.com'

In [21]:
from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route)

full_chain.invoke({"question": "I want to get the latest news on the stock market."})

'https://ww.x.com'

# Semantic Routing

In [24]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

physics_template = '''
You are a very smart physics teacher. \
You are great at answering questions about physics in a concise and clear easy way \
When you do not know the answer to a question you admit you do not know the answer.

Here is the question:
{query} 
'''

math_template = '''
You are a very good math teacher. You are great at answering math question because you able to break down hard problems into their component parts.

Answer the component parts and then put them together to answer the question.

Here is the question: {query}
'''

In [25]:
prompt_templates = [physics_template, math_template]

In [59]:
def embed_documents(documents: list[str]):
    embedded_documents = []
    for document in documents:
        encoded = emmbedding_model.encode(document)
        embedded_documents.append(encoded)
        
    return embedded_documents

In [35]:
prompt_embeddings = embed_documents(prompt_templates)

In [42]:
len(prompt_embeddings)

2

In [47]:
test_embedding = embed_documents(["hello world"])
test_embedding_another = emmbedding_model.encode("hello world")

In [50]:
sim = cosine_similarity([test_embedding_another], prompt_embeddings)
sim_another = cosine_similarity(test_embedding, prompt_embeddings)

In [58]:
cosine_similarity(
    [emmbedding_model.encode("I love chocolate")],
    embed_documents(["I love chocolate", "I hate chocolate"]))

array([[1.       , 0.8081372]], dtype=float32)

In [51]:
print(sim)
print(sim_another)

[[0.07782761 0.04691144]]
[[0.07782761 0.04691144]]


In [46]:
sim

array([[0.07782761, 0.04691144]], dtype=float32)

In [64]:
# route question
def prompt_router(input):
    # embed query
    query_embedding = embed_documents([input["query"]])
    prompt_embeddings = embed_documents(prompt_templates)
    
    similarity = cosine_similarity(query_embedding, prompt_embeddings)
    most_similar_index = similarity[0].argmax()
    most_similar = prompt_templates[most_similar_index]
    #
    print("Using MATH: " if most_similar == math_template else "Using PHYSICS: ")
    prompt = PromptTemplate.from_template(most_similar)
    return prompt
    

In [65]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | (lambda x : x.content)
)

chain

{
  query: RunnablePassthrough()
}
| RunnableLambda(prompt_router)
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002565766EFC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002565752ADE0>, model_kwargs={}, groq_api_key=SecretStr('**********'))
| RunnableLambda(...)

In [67]:
response = chain.invoke("What is the speed of light?")
print(response)

Using PHYSICS: 
The speed of light is approximately 299,792 kilometers per second, or about 186,282 miles per second. This is a fundamental constant in physics and is denoted by the letter c. It is the maximum speed at which information or matter can travel.


In [68]:
response = chain.invoke("What is pythagoras theorem?")
print(response)

Using PHYSICS: 
Pythagorean theorem states that in a right-angled triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides. This can be written as: a² + b² = c², where c represents the length of the hypotenuse, and a and b represent the lengths of the other two sides.


In [70]:
response = chain.invoke("What is force")
print(response)

Using PHYSICS: 
Force is a push or pull on an object resulting from the object's interaction with another object. It is a vector quantity, meaning it has both magnitude (size) and direction. Forces can cause an object to move, stop moving, or change direction. The unit of force in the International System of Units (SI) is the newton (N).


In [79]:
print(llm.invoke("what is force").content)

Force is a push or pull upon an object resulting from the object's interaction with another object. It is a vector quantity, meaning that it has both magnitude (size) and direction. Forces can cause an object to move, stop moving, or change its direction. They can also cause deformation or damage to an object. Forces can arise from various sources, such as gravity, friction, normal force, tension, and air resistance. The unit of force in the International System of Units (SI) is the newton (N), which is defined as the force required to accelerate a mass of one kilogram at a rate of one meter per second squared.
